In [1]:
import pandas as pd
import warnings
from pandas.core.common import SettingWithCopyWarning
from yahoo_fin import stock_info as si
import datetime
import pickle
import tqdm
import numpy as np
import pandas_market_calendars as mcal
import os
import emoji
import gspread
import df2gspread as d2g

In [2]:

gc = gspread.oauth(credentials_filename="client_secret_279764153689-2rq7k9683jqc57dh9o6lf30jvatdec7m.apps.googleusercontent.com.json",)


In [3]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
today = datetime.datetime.today().date()
s_date = '2021-12-31'
nyse = mcal.get_calendar('NYSE') # set up calendar
market_days = nyse.valid_days(start_date=s_date, end_date=today).date # get valid market days

In [5]:
def create_initials():
    """
    creates the initial allocations using the provided CSV
    :return: a pandas df of initial allocations
    """
    inital_allocs = pd.read_csv('assets/init_alloc.csv', index_col='stocks') # read in initial allc csv
    inital_allocs = inital_allocs*1000000 # convert to $ value
    stocks = list(inital_allocs.index) # put all stocks in list
    px_dat = [] # px data for day 0
    for stock in stocks:# loop through all stocks
        k = si.get_data(stock, start_date='2021-12-31', end_date='2022-01-01', interval='1d') # request relevant info
        px = k['close'].iloc[0] # take only the closing px on the day
        px_dat.append(px) # append to the list
    inital_allocs['px_dat'] = px_dat # add px data to alloc df
    init_shares = inital_allocs.floordiv(inital_allocs['px_dat'].to_numpy(), axis=0) # ensures
    init_shares['px_dat'] = px_dat
    init_shares.to_csv("assets/shares.csv", index = True)
    return init_shares

In [6]:
create_initials()

,Anish,Emily,Paul,Kirby,Hosegood,Leong,Olivia,Brokaw,Eric H,Kyle Cross,...,Farrah,Baldridge,Rubyann,Mac,Madison,Karsa,Sergio,Daniel K,Candice,px_dat
stocks,,,,,,,,,,,,,,,,,,,,,
RIVN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,964.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103.690002
LCID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2628.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.049999
APPS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1639.0,0.0,4099.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.990002
BUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9469.0,0.0,4734.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.560000
FIGS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3628.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.559999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIDU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,672.0,148.789993
BABA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,841.0,118.790001
DIDI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20080.0,4.980000


In [7]:
def update_stocklist(stocks_in=[], stocks_out=[]):
    """
    create a security master file
    :param stocks_in: stocks that are going into portfolios.
    :param stocks_out: Stocks comming out of portfolios
    :return:
    """
    try: #try to do the following
        with open('assets/stock_list.pkl', 'rb') as f: # open the stock list pickle file
            stock_list = pickle.load(f) # load the file
        new = [] # this is a new list of stocks
        for s in stock_list: # for every stock in the list of existing stocks
            if s not in stocks_out: # if the stock is not in the out list
                new.append(s) # keep the stock in the master stock list
        for t in stocks_in: # for every stock in the list of new stocks
            if t not in new: # if the stock is NOT in the master already
                new.append(t) # add it
        with open('assets/stock_list.pkl', 'wb') as f: # re-open the pickle file
            pickle.dump(new, f) # save it
    except: #otherwise
        with open('assets/stock_list.pkl', 'wb') as f: # just create a new master using the new assets
            pickle.dump(stocks_in, f) # save it
    return





In [8]:
def update_px():
    """
    update the prices of the stocks in the system
    :return:
    """
    with open('assets/stock_list.pkl', 'rb') as f: # open tha master stock list
        stock_list = pickle.load(f) # load into the system
    today = datetime.datetime.today().date().strftime('%Y-%m-%d') # set the date
    # for every stock in the system get stock price data and save to a csv
    for stock in tqdm.tqdm(stock_list):
        k = si.get_data(stock, start_date='2021-12-31', end_date=today, interval='1d') # request relevant info
        k.to_csv('assets/px_dat/{}_px.csv'.format(stock))
    return

In [9]:
def update_bm():
    """
    update the benchmark data
    :return:
    """
    with open('assets/bm_list.pkl', 'rb') as f: # open the BM master list
        stock_list = pickle.load(f) # load into the system
    today = datetime.datetime.today().date().strftime('%Y-%m-%d') # today's date
    #for every stock in the system get stock price data and save to a csv
    for stock in tqdm.tqdm(stock_list):
        k = si.get_data(stock, start_date='2021-12-31', end_date=today, interval='1d') # request relevant info
        k.to_csv('assets/bm_dat/{}_px.csv'.format(stock))
    return

In [10]:
def update_divs():
    """
    update dividend data for all stocks
    :return:
    """
    with open('assets/stock_list.pkl', 'rb') as f: # open tha master stock list
        stock_list = pickle.load(f) # load into the system
    today = datetime.datetime.today().date().strftime('%Y-%m-%d') # set the date
   # for every stock in the system get stock dividend data and save to a csv
    for stock in tqdm.tqdm(stock_list):
        k = si.get_dividends(stock, start_date='2021-12-31', end_date=today) # request relevant info
        k.to_csv('assets/dividends/{}_div.csv'.format(stock))
    return

In [11]:
update_px()

100%|██████████| 146/146 [00:19<00:00,  7.55it/s]


In [12]:
update_divs()

100%|██████████| 146/146 [00:18<00:00,  7.81it/s]


In [13]:
update_bm()


100%|██████████| 8/8 [00:01<00:00,  7.62it/s]


In [14]:
def generate_snap(name, date, trades = {}, cash = 1000000):
    """
    creates a portfolio appraisal
    :param name: <str> name of the player
    :param date: <str> the date in question format is YYYY-MM-DD
    :param trades: <dict> list of trades to execute
    :return: <df> appraisal
    """
    # initial trade
    if date == '2021-12-31': # assuming its the initial trade day:
        position = pd.DataFrame(trades, index=["Shares"]).T # Create the initial positions
        px = [pd.read_csv('assets\px_dat\{}_px.csv'.format(i), index_col=0).loc[date]['close'] for i in position.index] # read the price data
        position['Cost Per Share'] = px # add price data to the initial positions df
        position['Cost Basis'] = position['Cost Per Share'] * position['Shares'] # calculate cost basis
        position['Current Price'] = px # current px is also the close in this case
        position['As of {}'.format(date)] = position['Current Price'] * position['Shares'] # calculate currenc vlaue
        position['Return'] = (position['As of {}'.format(date)]/position['Cost Basis'])-1 # calculate position returns
        divs = [] # set up dividend list
        # get dividends
        for stock in position.index: # loop through stocks
            d_dat = pd.read_csv('assets\dividends\{}_div.csv'.format(stock), index_col=0) # get div data
            d_dat.index = pd.to_datetime(d_dat.index) # set dates as datetime
            d_dat = d_dat[d_dat.index==pd.to_datetime(date)] # isolate today's date
            if len(d_dat)>0: # if dividend data exists
                divs.append(d_dat['dividend'].loc[pd.to_datetime(date)]) # add it to the divs list
            else:
                divs.append(0) # otherwise just add 0
        div_pay = np.dot(position['Shares'], np.array(divs))
        new_cash = cash - np.sum(position['As of {}'.format(date)]) + div_pay #  calculate the new cash position
        cash_pos = pd.DataFrame({"Shares":new_cash, "Cost Per Share": 1, "Cost Basis" : new_cash,
                                 "Current Price": 1, 'As of {}'.format(date): new_cash, "Return":0}, index = ["Cash"]) # create cash line
        position = pd.concat([position, cash_pos]) # combine cash line with positions
        total_line = pd.DataFrame({'Cost Basis': position['Cost Basis'].sum(),
                                   'As of {}'.format(date): position['As of {}'.format(date)].sum()}, index = ["Total"])# create total line
        total_line['Return'] = (total_line['As of {}'.format(date)]/total_line['Cost Basis'])-1 # calculate portfolio total return (like this for day one only)
        position = pd.concat([position, total_line]) # combine total with positions
        position = position.fillna("") # fill all nas with white space
        position.to_csv('assets/port_apps/{}_apr_{}.csv'.format(name, date)) # create csv
    else:
        loc = np.where(market_days== pd.to_datetime(date))[0][0] # finde the location of todays date in the market days calendar
        previous_loc = loc-1 # go back one day
        prev_date = market_days[previous_loc].strftime("%Y-%m-%d") # get yesterday's date
        position = pd.read_csv("assets/port_apps/{}_apr_{}.csv".format(name, prev_date),index_col = 0) # get the previous apraisal
        previous_cash = position.loc['Cash']['As of {}'.format(prev_date)] # store the cash value
        previous_port_val = position['As of {}'.format(prev_date)].iloc[:-1].sum()
        px = [pd.read_csv('assets\px_dat\{}_px.csv'.format(i), index_col=0).loc[date]['close'] for i in position.index[:-2]] # read the price data
        px.append(1) #append for cash
        px.append('') # append for total
        position['Current Price'] = px # current px
        position['As of {}'.format(date)] = position['Current Price'] * position['Shares'] # calculate currenc vlaue
        position['Return'] = (position['As of {}'.format(date)]/position['Cost Basis'])-1 # calculate position returns
        divs = [] # set up dividend list
        # get dividends
        for stock in position.index[:-2]: # loop through stocks
            d_dat = pd.read_csv('assets\dividends\{}_div.csv'.format(stock), index_col=0) # get div data
            d_dat.index = pd.to_datetime(d_dat.index) # set dates as datetime
            d_dat = d_dat[d_dat.index==pd.to_datetime(date)] # isolate today's date
            if len(d_dat)>0: # if dividend data exists
                divs.append(d_dat['dividend'].loc[pd.to_datetime(date)]) # add it to the divs list
            else:
                divs.append(0) # otherwise just add 0
        div_pay = np.dot(position['Shares'].iloc[:-2], np.array(divs))
        new_cash = previous_cash+div_pay
        if len(trades.keys())>0:
            new_position = pd.DataFrame(trades, index=["Shares"]).T # Create the initial positions
            px_new = [pd.read_csv('assets\px_dat\{}_px.csv'.format(i), index_col=0).loc[date]['close'] for i in new_position.index] # read the price data
            new_position['Cost Per Share'] = px_new # add price data to the initial positions df
            new_position['Cost Basis'] = new_position['Cost Per Share'] * new_position['Shares'] # calculate cost basis
            new_position['Current Price'] = px_new # current px is also the close in this case
            new_position['As of {}'.format(date)] = new_position['Current Price'] * new_position['Shares'] # calculate currenc vlaue
            if new_cash - new_position['As of {}'.format(date)].sum() > 0: # make sure new cash is greater than 0
                new_cash = new_cash - new_position['As of {}'.format(date)].sum() # adjust cash for new positions
                new_apr = pd.concat([position.iloc[:-2], new_position]) # combine old positions with new
                new_apr = new_apr.drop(['As of {}'.format(prev_date)], axis = 1) # remove old value
                updated_apr = pd.DataFrame() # this is the new appraisal df
                for stock in new_apr.index.unique(): # loop through the stocks
                    df = new_apr[new_apr.index==stock] # isolate stock lines
                    if len(df)<2: # if there is only one line just send that to the new df
                        updated_apr = pd.concat([updated_apr, df])
                    else:
                        post_ex_shares = df['Shares'].sum() # add shares
                        post_ex_cost_basis = df['Cost Basis'].sum() # combine cost basis
                        post_ex_as_of = df['As of {}'.format(date)].sum() # combine values
                        post_ex_cost_per = post_ex_cost_basis/post_ex_shares # back out per share cost
                        post_ex_cur = post_ex_as_of/post_ex_shares # share value
                        to_apr = pd.DataFrame({"Shares":post_ex_shares, "Cost Per Share":post_ex_cost_per,
                                               "Cost Basis": post_ex_cost_basis, "Current Price": post_ex_cur,
                                               'As of {}'.format(date): post_ex_as_of}, index=[stock]) # create the df line
                        updated_apr = pd.concat([updated_apr, to_apr]) # add the new position to the apr
                position = updated_apr.copy(deep = True) # this is now the new position
            else:
                pass
        else:
            position = position.iloc[:-2]
            position = position.drop(['As of {}'.format(prev_date)], axis = 1)
        position['Return'] = (position['As of {}'.format(date)]/position['Cost Basis'])-1 # calculate position returns
        cash_pos = pd.DataFrame({"Shares":new_cash, "Cost Per Share": 1, "Cost Basis" : new_cash,
                                 "Current Price": 1, 'As of {}'.format(date): new_cash, "Return":0}, index = ["Cash"]) # create cash line
        position = pd.concat([position, cash_pos]) # combine cash line with positions
        total_line = pd.DataFrame({'Cost Basis': position['Cost Basis'].sum(),
                                   'As of {}'.format(date): position['As of {}'.format(date)].sum()}, index = ["Total"])# create total line
        total_line['Return'] = (total_line['As of {}'.format(date)]/1000000)-1 # calculate portfolio total return (like this for day one only)
        position = pd.concat([position, total_line]) # combine total with positions
        position = position.fillna("") # fill all nas with white space
        position.to_csv('assets/port_apps/{}_apr_{}.csv'.format(name, date)) # create csv
        position= position.reset_index()
        sh = gc.open("sc_aps")
        try:
            worksheet = sh.worksheet('{}_apr'.format(name))
        except:
            worksheet = sh.add_worksheet('{}_apr'.format(name), 100, 100)
        worksheet.update([position.columns.values.tolist()] + position.values.tolist())


    return position



In [22]:
# update performance
def update_perf(name):
    apr_path = "assets/port_apps" # path to appraisal file
    apr_list = os.listdir(apr_path) # create list of apraisals
    relev_files = [i for i in apr_list if i.split("_")[0]==name]
    dates = []
    values = []
    for file in relev_files:
        df = pd.read_csv('{}/{}'.format(apr_path,file), index_col=0)
        date = file.split("_")[-1]
        date = date.split(".")[0]
        dates.append(date)
        val = df["As of {}".format(date)].loc['Total']
        values.append(val)
    perf_file = pd.DataFrame({"Date":dates, "Value": values})
    perf_file['Daily Return']=perf_file['Value'].pct_change().fillna(0)
    perf_file['Total Return'] = (perf_file['Value']/perf_file['Value'].iloc[0])-1
    bm_key = pd.read_csv("assets/Player_bm.csv", index_col=0)
    bm = bm_key[bm_key["Name"]==name]["Benchmark"].iloc[0]
    bm_px = pd.read_csv("assets/bm_dat/{}_px.csv".format(bm), index_col=0)
    bm_px = bm_px.filter(['adjclose'])
    bm_px['{} Daily Return'.format(bm)] = bm_px['adjclose'].pct_change().fillna(0)
    bm_px['{} Total Return'.format(bm)] = (bm_px['adjclose']/bm_px['adjclose'].iloc[0])-1
    perf_file=perf_file.set_index('Date').join(bm_px)
    perf_file.to_csv("assets/performance/{}_perf.csv".format(name))
    sh = gc.open("sc_perf")
    try:
        worksheet = sh.worksheet('{}_perf'.format(name))
    except:
        worksheet = sh.add_worksheet('{}_perf'.format(name), 500, 500)
    worksheet.update([perf_file.columns.values.tolist()] + perf_file.values.tolist())



    return perf_file


In [29]:
def update_league_tab():
    path = "assets/performance"
    perf_list = os.listdir(path)
    names = []
    one_day = []
    five_day = []
    one_month = []
    qtr = []
    s_inc = []
    bm_names = []
    bm = []
    for doc in perf_list:
        names.append(doc.split('_')[0])
        rets = pd.read_csv("{}/{}".format(path,doc))
        bm_name= rets.columns[-1].split(" ")[0]
        bm_names.append(bm_name)
        one_day.append(rets['Daily Return'].iloc[-1])
        try:
            fiv = np.prod(rets['Daily Return'].iloc[-5:] + 1)-1
        except:
            fiv = np.prod(rets['Daily Return'] + 1)-1
        try:
            mth = np.prod(rets['Daily Return'].iloc[-21:] + 1)-1
        except:
            mth = np.prod(rets['Daily Return'] + 1)-1
        try:
            qt = np.prod(rets['Daily Return'].iloc[-63:] + 1)-1
        except:
            qt = np.prod(rets['Daily Return'] + 1)-1
        si = np.prod(rets['Daily Return'] + 1)-1
        bms = rets['{} Total Return'.format(bm_name)].iloc[-1]
        five_day.append(fiv)
        one_month.append(mth)
        qtr.append(qt)
        s_inc.append(si)
        bm.append(bms)
    league = pd.DataFrame({"Player":names, "1-Day":one_day, "5-Day":five_day, "One Month":one_month,
                          "One Quarter":qtr, "Since Inception":s_inc, "Benchmark Ticker":bm_names,"Benchmark Return":bm})
    league = league.sort_values(['Since Inception'], ascending=False)
    stars = [emoji.emojize(":money_with_wings:", language='alias') for i in range(0,5)]
    poop = [emoji.emojize(":poop:", language='alias') for i in range(0,5)]
    unam = [emoji.emojize(":unamused:", language='alias') for i in range(0,16)]
    status = stars+unam+poop
    league['Status'] = status
    league = league.reset_index(drop=True)
    date = rets['Date'].iloc[-1]
    league.to_csv("league_{}.csv".format(date))
    sh = gc.open("league")
    try:
        worksheet = sh.worksheet('league')
    except:
        worksheet = sh.add_worksheet('league', 500, 500)
    worksheet.update([league.columns.values.tolist()] + league.values.tolist())


    return league

update_league_tab()


,Player,1-Day,5-Day,One Month,One Quarter,Since Inception,Benchmark Ticker,Benchmark Return,Status
0,Melinda,-0.003304,0.019855,0.050736,0.053567,0.053567,VLUE,-0.022994,💸
1,Westy,0.020975,0.037286,0.069073,0.015641,0.015641,IVW,-0.074205,💸
2,Eric H,0.006879,0.033826,0.098538,-0.031299,-0.031299,IVV,-0.036981,💸
3,Kirby,0.023136,0.060150,0.125780,-0.038493,-0.038493,IVW,-0.074205,💸
4,Olivia,0.019563,0.021085,0.023126,-0.043978,-0.043978,IVW,-0.074205,💸
5,Karsa,0.020984,0.043831,-0.029643,-0.046101,-0.046101,IYC,-0.098082,😒
6,Nour,0.001744,0.026117,0.042066,-0.059499,-0.059499,IVW,-0.074205,😒
7,AK,0.006251,0.007449,0.041561,-0.071931,-0.071931,MTUM,-0.061115,😒
8,Rubyann,0.012886,0.009734,-0.001542,-0.078154,-0.078154,IVW,-0.074205,😒
9,Daniel K,0.014348,0.028716,0.026642,-0.097813,-0.097813,QUAL,-0.070335,😒


In [25]:
# create_updates
port_shares = pd.read_csv('assets/shares.csv',index_col=0) # read the shares csv into the system
date = market_days[-2]
date = date.strftime("%Y-%m-%d")
for name in port_shares.columns[:-1]:
    generate_snap(name=name, date=date, cash=0)

for name in port_shares.columns[:-1]:
    update_perf(name)


In [66]:
#id_bms for all
#names = [name for name in port_shares.columns[:-1]]
#bms = [id_bm(name) for name in port_shares.columns[:-1]]
#bm_df = pd.DataFrame({"Name": names, "Benchmark":bms})
#bm_df.to_csv("assets/Player_bm.csv")


In [ ]:
def id_bm(name):
    """
    gets benchmark for port
    """
    port_shares = pd.read_csv('assets/shares.csv',index_col=0)  # read in portfolio data
    bm_dat = pd.read_csv('assets/bm_dat.csv', index_col = 0) # read bm_data
    port = port_shares[name].to_frame() # isolate the portfolio in question
    port = port[port[name]!=0] # eliminate every row that equals 0
    holdings = set(port.index) # create a set of holdings
    overlap = [] # overlaping list
    for b in bm_dat.columns[:-1]: # go through the benchmarks and look for overlaps
        stk = bm_dat[b].to_frame()
        stk = stk[stk[b]!=0]
        b_hold = set(stk.index)
        over = holdings & b_hold
        overlap.append((b, len(over)))
    overlap.sort(key=lambda x: x[1], reverse = True) # sort the overlap list by most to least overlap
    finalists = overlap[0:1] # chose finalists
    # chose the etf in question as a bm.
    if finalists[0][1]== 0:
        bm = bm_dat.columns[-1]
    else:
        bm = finalists[0][0]



    return bm

In [59]:
id_bm("Anish")

'IVW'

['QUAL', 'MTUM', 'VLUE', 'IWM', 'IVW', 'IYW', 'IYC', 'IVV']

,stock,QUAL,MTUM,VLUE,IWM,IVW,IYW,IYC,IVV
0,T,0.00000,0.00000,7.19575,0.00000,0.00000,0.00000,0.00000,0.46190
1,INTC,1.18210,0.00000,5.64419,0.00000,0.00000,1.93978,0.00000,0.49526
2,MU,0.00000,0.00000,4.03235,0.00000,0.00000,1.08384,0.00000,0.27464
3,CSCO,1.66228,0.00000,3.68796,0.00000,0.59213,0.00000,0.00000,0.65348
4,F,0.00000,1.00559,3.04572,0.00000,0.00000,0.00000,1.33617,0.19237
...,...,...,...,...,...,...,...,...,...
3010,CHWY,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3011,USD,0.00000,0.00000,0.00000,0.05323,0.04958,0.05267,0.02993,0.11133
3012,SOFI,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3013,LCID,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [70]:

bm_key

,Name,Benchmark
0,Anish,IVW
1,Emily,IYW
2,Paul,IVW
3,Kirby,IVW
4,Hosegood,IWM
5,Leong,IYC
6,Olivia,IVW
7,Brokaw,IVW
8,Eric H,IVV
9,Kyle Cross,IWM


In [51]:
print(emoji.emojize(':poop:'))

:poop:


In [52]:
df = pd.read_csv("assets/performance/Anish_perf.csv")
df

,Date,Value,Daily Return,Total Return,adjclose,IVW Daily Return,IVW Total Return
0,2021-12-31,1.000000e+06,0.000000,0.000000,83.528221,0.000000,0.000000
1,2022-01-03,1.016174e+06,0.016174,0.016174,84.177124,0.007769,0.007769
2,2022-01-04,1.002549e+06,-0.013408,0.002549,83.338554,-0.009962,-0.002271
3,2022-01-05,9.689505e+05,-0.033513,-0.031049,80.802856,-0.030426,-0.032628
4,2022-01-06,9.797765e+05,0.011173,-0.020223,80.573242,-0.002842,-0.035377
5,2022-01-07,9.778325e+05,-0.001984,-0.022168,79.794563,-0.009664,-0.044699
6,2022-01-10,9.735195e+05,-0.004411,-0.026480,79.854462,0.000751,-0.043982
7,2022-01-11,9.882171e+05,0.015097,-0.011783,80.722984,0.010876,-0.033584
8,2022-01-12,9.822882e+05,-0.006000,-0.017712,81.112328,0.004823,-0.028923
9,2022-01-13,9.647524e+05,-0.017852,-0.035248,79.085770,-0.024985,-0.053185


port = pd.read_csv('assets/port_apps/{}_{}'.format(name, date))

In [53]:
port = pd.read_csv('assets/port_apps/{}_apr_{}.csv'.format('Anish', date))

In [54]:
port = port.fillna(" ")

In [58]:
apr_db = gc.create('sc_aps')
perf_db = gc.create('sc_perf')

In [56]:
worksheet = sh.add_worksheet("test", 100, 100)

In [28]:
lg_db = gc.create('league')

In [57]:
worksheet.update([port.columns.values.tolist()] + port.values.tolist(),)

{'spreadsheetId': '13gSQL3XfW8p7SDfj_Lv6eWM37gDcVtpkKmb7bY0_WcA',
 'updatedRange': 'test!A1:G13',
 'updatedRows': 13,
 'updatedColumns': 7,
 'updatedCells': 91}